In [1]:
#Importación de librerias útiles
import pandas
corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')

#Librerias cientificas scipy, numpy
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time
import math

#Lectura de archivos xml
from lxml import etree

#Expresiones regulares
import re

#Pyfreeling
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Para aplicar Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizer = CountVectorizer(stop_words='english')

## Eliminar comentarios con menos de tres votos

In [18]:
#Listas para salvar los tweets filtrados
id_f = []
text_f = []
account_id_f=[]
nh_f=[]
sh1_f=[]
sh2_f=[]
sh3_f=[]
sh4_f=[]
sh5_f=[]

#Listas del corpus
vec_id=corpus['id'][:]
vec_text=corpus['text'][:]
vec_account_id=corpus['account_id'][:]
vec_no_humor =corpus['n'][:]
vec_e1 =corpus['1'][:]
vec_e2 =corpus['2'][:]
vec_e3 =corpus['3'][:]
vec_e4 =corpus['4'][:]
vec_e5 =corpus['5'][:]

for i in range(len(corpus)):
    if vec_no_humor[i] + vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] >= 3:
        id_f.append(vec_id[i])
        text_f.append(vec_text[i])
        account_id_f.append(vec_account_id[i])
        nh_f.append(vec_no_humor[i])
        sh1_f.append(vec_e1[i])
        sh2_f.append(vec_e2[i])
        sh3_f.append(vec_e3[i])
        sh4_f.append(vec_e4[i])
        sh5_f.append(vec_e5[i])
        



In [35]:
print("Tweets con al menos tres votos", len(text_f))
d = {'id' : id_f,
    'text' : text_f,
    'account_id': account_id_f,
    'n':nh_f, 
    '1':sh1_f,
    '2':sh2_f,
    '3':sh3_f,
    '4':sh4_f,
    '5':sh5_f
    }
df = pandas.DataFrame(d, columns = ['id', 'text', 'account_id', 'n', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro1.csv')
print("Tweets guardados en corpus_filtro1.csv")

Tweets con al menos tres votos 3438
Tweets guardados en corpus_filtro1.csv


## La estructura con la que se va a trabajar ahora se levanta del archivo corpus_filtro1.cvs, que contiene solo los tweets con a los sumo tres votos

# La siguiente actividad es eliminar Hashtags

In [59]:
text_f = []

corpus_filtro1 = pandas.read_csv("corpus_filtro1.csv",encoding='utf-8')

pattern_hashtag = re.compile(r'#.+?\b')



for i in range(len(corpus_filtro1)):
    text_f.append(re.sub(pattern_hashtag, "", corpus_filtro1['text'][i]))


In [60]:
print("Tweets sin hashtags")
d = {'id' : corpus_filtro1['id'][:],
    'text' : text_f,
    'account_id': corpus_filtro1['account_id'][:],
    'n':corpus_filtro1['n'][:], 
    '1':corpus_filtro1['1'][:],
    '2':corpus_filtro1['2'][:],
    '3':corpus_filtro1['3'][:],
    '4':corpus_filtro1['4'][:],
    '5':corpus_filtro1['5'][:]
    }
df = pandas.DataFrame(d, columns = ['id', 'text', 'account_id', 'n', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro2.csv')
print("Tweets guardados en corpus_filtro2.csv")

Tweets sin hashtags
Tweets guardados en corpus_filtro2.csv


# La estructura con la que se va a trabajar ahora se levanta desde corpus_filtro2.csv. Esta estructura contiene los tweets que tienen al menos tres votos y se han eliminado del texto los hashtags.

# A continuación se agregará al corpus la información de si es humorístico o no y manteniendo solo los campos 1,2,3,4,5

In [2]:
corpus_filtro2 = pandas.read_csv("corpus_filtro2.csv",encoding='utf-8')

#Listas para salvar los tweets filtrados
text = []
category=[]
h1=[]
h2=[]
h3=[]
h4=[]
h5=[]

#Listas del corpus
vec_text=corpus_filtro2['text'][:]
vec_no_humor =corpus_filtro2['n'][:]
vec_e1 =corpus_filtro2['1'][:]
vec_e2 =corpus_filtro2['2'][:]
vec_e3 =corpus_filtro2['3'][:]
vec_e4 =corpus_filtro2['4'][:]
vec_e5 =corpus_filtro2['5'][:]

for i in range(len(corpus_filtro2)):
    if  vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] >= vec_no_humor[i]:
        category.append(1)
    else:
        category.append(0)
    text.append(vec_text[i])
    h1.append(vec_e1[i])
    h2.append(vec_e2[i])
    h3.append(vec_e3[i])
    h4.append(vec_e4[i])
    h5.append(vec_e5[i])

In [3]:
print("Guardando corpus y categorias")
d = {'text' : text,
    'category': category,
     '1':h1,
     '2':h2,
     '3':h3,
     '4':h4,
     '5':h5
    }
df = pandas.DataFrame(d, columns = ['text','category','1','2','3','4','5'])
df.to_csv('corpus_filtro3.csv')
print("Tweets guardados en corpus_filtro3.csv")

Guardando corpus y categorias
Tweets guardados en corpus_filtro3.csv


# Se agregará información de POS-tag al texto, para considerar las POS-tag como features 

In [ ]:
corpus_filtro3 = pandas.read_csv("corpus_filtro3.csv",encoding='utf-8')
text_list=corpus_filtro3['text'][:]

pattern_pos = re.compile(r'pos="(.*?)"')

numsentence=1
pos=0
for d in text_list:
    xml = analyzer.run(d.encode(), 'flush')
    print(numsentence, end="\r")
    for sentence in xml:        
        for token in sentence:
            m = re.search(pattern_pos, etree.tostring(token).decode())
            if m is not None:
                text_list[pos] = text_list[pos] + " " + m.group(1)
    pos+=1
    numsentence+=1

In [74]:
print("Guardando corpus y categorias")
d = {'text' : text_list,
    'category': category[:5]
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro4.csv')
print("Tweets guardados en corpus_filtro4.csv")

Guardando corpus y categorias
Tweets guardados en corpus_filtro4.csv


# Separación de datos de entrenamiento y datos de desarrollo

In [76]:
corpus_filtro4 = pandas.read_csv("corpus_filtro4.csv",encoding='utf-8')
num_dev_tweets=math.floor(len(corpus_filtro4)*20/100)
random_samples=np.random.randint(0, len(corpus_filtro4) - 1, size=num_dev_tweets)

print(num_dev_tweets)
print(random_samples)

#Listas para entrenamiento
text_train = []
category_train=[]

#Listas para desarrollo
text_dev = []
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4['text'][:5]
vec_category=corpus_filtro4['category'][:5]

for i in range( len(corpus_filtro4)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        category_dev.append(vec_category[i])
    else:
        text_train.append(vec_text[i])
        category_train.append(vec_category[i])
        
print("Cantidad de tweets para entrenamiento", len(text_train))
print("Cantidad de tweets para desarrollo", len(text_dev))


1
[0]
Cantidad de tweets para entrenamiento 4
Cantidad de tweets para desarrollo 1


In [77]:
print("Guardando corpus y categorias para los conjuntos de entrenamiento y desarrollo")
d = {'text' : text_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_trainingset.csv')
print("Tweets guardados en corpus_filtro5_trainingset.csv")
d = {'text' : text_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_devset.csv')
print("Tweets guardados en corpus_filtro5_devset.csv")

Guardando corpus y categorias para los conjuntos de entrenamiento y desarrollo
Tweets guardados en corpus_filtro5_trainingset.csv
Tweets guardados en corpus_filtro5_devset.csv


# Prediccion de tweets naive bayes

In [80]:
vectorizer = CountVectorizer(stop_words='english')

In [81]:
corpus_filtro5_trainingset = pandas.read_csv("corpus_filtro5_trainingset.csv",encoding='utf-8')
text_list=corpus_filtro5_trainingset['text'][:]
category_list=corpus_filtro5_trainingset['category'][:]
train_features = vectorizer.fit_transform(text_list)

In [82]:
corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
text_devlist=corpus_filtro5_devset['text'][:]
category_devlist=corpus_filtro5_devset['category'][:]
test_features = vectorizer.transform(text_devlist)


In [83]:
nb = MultinomialNB()
nb.fit(train_features , category_list)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [90]:
predictions = nb.predict(test_features)

In [94]:
from sklearn.naive_bayes import MultinomialNB
print(predictions)
nb.predict_proba(test_features)

[1]


array([[ 0.02213721,  0.97786279]])